In [22]:
# Read libraries

# Data wrangling
import pandas as pd
import numpy as np
import re
from shapely import wkt
import fiona

# Visualization 
import missingno as msno
import matplotlib
import mapclassify
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import seaborn as sns
import geopandas as gpd


# Geostatistics
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import libpysal as ps
import matplotlib.pyplot as plt
import seaborn as sns
from pysal.lib import weights
from pysal.model import spreg
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.compat import lzip
from esda.moran import Moran_Local
from splot.esda import lisa_cluster
from pysal.lib import weights
from splot.libpysal import plot_spatial_weights
from spreg import OLS
from spreg import MoranRes
from spreg import ML_Lag
from spreg import ML_Error

# Machine learning 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import mutual_info_regression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.ensemble import RandomForestRegressor

# Look at this libraries later
#from keras.models import Sequential
#from keras.layers import Dense, Dropout
#from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the GeoParquet file without one-hot sampling
gdf_no_oh = gpd.read_parquet('gdf_no_ohe.parquet')

In [3]:
gdf_no_oh

,year,countries,region,pf_ss_disappearances_disap,pf_ss_disappearances_violent,pf_ss_disappearances_violent_data,pf_ss_disappearances_organized,pf_ss_disappearances_fatalities,pf_ss_disappearances_fatalities_data,pf_ss_disappearances_injuries,...,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,country_code,geometry,globalNorth,ef_score,hf_score,pf_expression_bti,pf_rol_civil
0,2020.0,Albania,Eastern Europe,10.0,10.000000,0.0,7.500000,10.000000,0.0,10.000000,...,7.175250,6.698118,7.112958,ALB,"POLYGON ((20.07142 42.56091, 20.10208 42.53347...",1.0,0,0,0,0
1,2020.0,Algeria,Middle East & North Africa,10.0,9.687083,25.0,5.000000,10.000000,0.0,10.000000,...,7.029528,5.686753,5.778953,DZA,"POLYGON ((8.62203 36.94137, 8.63222 36.88194, ...",0.0,0,0,0,0
2,2020.0,Angola,Sub-Saharan Africa,10.0,9.582498,25.0,7.500000,9.736578,5.0,9.971733,...,6.782923,5.696942,6.227545,AGO,"MULTIPOLYGON (((23.98621 -10.87046, 23.98805 -...",0.0,0,0,0,0
3,2020.0,Argentina,Latin America & the Caribbean,5.0,10.000000,0.0,7.500000,9.925379,2.0,10.000000,...,6.508295,6.225232,5.490538,ARG,"MULTIPOLYGON (((-68.64312 -54.88861, -68.63723...",0.0,0,0,0,0
4,2020.0,Armenia,Caucasus & Central Asia,10.0,10.000000,0.0,7.500000,10.000000,0.0,10.000000,...,7.040738,7.124727,7.756333,ARM,"POLYGON ((46.54038 38.87559, 46.51639 38.87804...",1.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,2000.0,Venezuela,Latin America & the Caribbean,10.0,10.000000,0.0,8.231718,10.000000,0.0,10.000000,...,0.315142,4.692328,5.531397,VEN,"MULTIPOLYGON (((-61.18507 8.49674, -61.19417 8...",0.0,0,0,0,1
3482,2000.0,Vietnam,South Asia,10.0,10.000000,0.0,7.964260,10.000000,0.0,10.000000,...,0.000000,3.861879,4.416768,VNM,"MULTIPOLYGON (((104.08288 10.36486, 104.08663 ...",0.0,0,0,0,1
3483,2000.0,Yemen,Middle East & North Africa,5.0,10.000000,0.0,7.044853,8.170079,19.0,7.932996,...,6.988352,5.658095,6.684552,YEM,"MULTIPOLYGON (((52.23416 12.20111, 52.27888 12...",0.0,1,1,0,1
3484,2000.0,Zambia,Sub-Saharan Africa,10.0,10.000000,0.0,7.552838,10.000000,0.0,9.827262,...,8.520369,8.020982,6.799421,ZMB,"POLYGON ((32.94040 -9.40508, 32.93944 -9.41583...",0.0,0,0,0,1


## Feature scaling

Feature scaling is a method used to standardize the range of independent variables or features of data. In data processing, it is also known as data normalization and is generally performed during the data preprocessing step.

Note: Before doing feature scaling, make sure to split your data into train and test sets, to avoid data leakage. Data leakage can happen when information from outside your training dataset is used to create the model. This can cause the model to overestimate its accuracy.

First, let's do a **Train-Test Split** with respect to the temporal data nature:

In [4]:
# Sort the data by year
gdf_no_oh = gdf_no_oh.sort_values('year')

# Create the X and y variables
X = gdf_no_oh.drop(['ef_score', 'hf_score', 'pf_expression_bti', 'pf_rol_civil'], axis=1)  # Features
y = gdf_no_oh[['ef_score', 'hf_score', 'pf_expression_bti', 'pf_rol_civil']]  # Targets

# Calculate the index at which to split the data
split_index = int(len(X) * 0.7)

# Perform a chronological train-test split
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [5]:
# Select only numeric features, excluding 'year'
X_train_numeric = X_train.select_dtypes(include=np.number).drop('year', axis=1)
X_test_numeric = X_test.select_dtypes(include=np.number).drop('year', axis=1)

# Initialize a new scaling object for normalizing input variables
scaler = StandardScaler()

# Fit the scaler object to the training data
scaler.fit(X_train_numeric)

# Apply the scaler object to the training and test sets
X_train_scaled = scaler.transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)

# Replace the original numeric columns with the scaled ones
X_train[X_train_numeric.columns] = X_train_scaled
X_test[X_test_numeric.columns] = X_test_scaled

In [13]:
X_train_numeric.columns

Index(['pf_ss_disappearances_disap', 'pf_ss_disappearances_violent',
       'pf_ss_disappearances_violent_data', 'pf_ss_disappearances_organized',
       'pf_ss_disappearances_fatalities',
       'pf_ss_disappearances_fatalities_data', 'pf_ss_disappearances_injuries',
       'pf_ss_disappearances_injuries_data', 'pf_ss_disappearances', 'pf_ss',
       'pf_expression_direct_killed', 'pf_expression_direct_killed_data',
       'pf_expression_direct_jailed', 'pf_expression_direct_jailed_data',
       'pf_expression_direct', 'pf_expression_cld', 'pf_identity_divorce',
       'pf_identity_fgm', 'ef_government_tax_income_data',
       'ef_government_tax_payroll_data', 'ef_government_soa', 'ef_government',
       'ef_legal_protection', 'ef_legal_military', 'ef_legal_integrity',
       'ef_legal_enforcement', 'ef_legal_regulatory', 'ef_legal_police',
       'ef_gender', 'ef_money_sd', 'ef_money_sd_data', 'ef_money_inflation',
       'ef_money_inflation_data', 'ef_money_currency', 'ef_money',
  

In [33]:
y_train.columns

Index(['ef_score', 'hf_score', 'pf_expression_bti', 'pf_rol_civil'], dtype='object')

Feature selection: Recursive Feature Elimination

** Note; same 10 for all, not good

In [21]:
# Initialize a logistic regression model
model = LogisticRegression()

# List to store selected features and their coefficients for each target
selected_features_list = []

# Perform RFE for each target
for target in y_train.columns:
    # Initialize RFE
    rfe = RFE(estimator=model, n_features_to_select=10)  # Adjust the number of features to select as necessary

    # Fit RFE to the scaled numeric data
    rfe.fit(X_train_numeric, y_train[target])

    # Get the features selected by RFE
    selected_features = X_train_numeric.columns[rfe.support_]

    # Get the coefficients for the selected features
    feature_stats = []
    for feature in selected_features:
        model.fit(X_train_numeric[[feature]], y_train[target])
        coefficient = model.coef_[0][0]
        feature_stats.append((feature, coefficient))

    # Append the selected features and their coefficients to the list
    selected_features_list.append(feature_stats)

# Print the names and coefficients of the final selected features for each target
for i, target in enumerate(y_train.columns):
    print(f"Target: {target}")
    for feature, coefficient in selected_features_list[i]:
        print(f"Feature: {feature}, Coefficient: {coefficient}")
    print()
    
# Take the union of all selected features
final_selected_features = set().union(*selected_features_list)

# Print the names of the final selected features
print(final_selected_features)

Target: ef_score
Feature: pf_expression_cld, Coefficient: -0.33923698425677323
Feature: ef_government, Coefficient: -0.4639055288680121
Feature: ef_legal_military, Coefficient: -0.18783925262137283
Feature: ef_gender, Coefficient: -2.440078738657847
Feature: ef_money_currency, Coefficient: 0.051533638212198994
Feature: ef_trade_regulatory_nontariff, Coefficient: -0.009488006446295362
Feature: ef_regulation_labor_firing, Coefficient: -0.02273354521283719
Feature: ef_regulation_business_burden, Coefficient: 0.06672047108395089
Feature: ef_regulation_business_impartial, Coefficient: -0.3516145375879384
Feature: globalNorth, Coefficient: -1.225823383178556

Target: hf_score
Feature: pf_expression_cld, Coefficient: -0.33923698425677323
Feature: ef_government, Coefficient: -0.4639055288680121
Feature: ef_legal_military, Coefficient: -0.18783925262137283
Feature: ef_gender, Coefficient: -2.440078738657847
Feature: ef_money_currency, Coefficient: 0.051533638212198994
Feature: ef_trade_regulato

Feature selection: Mutual Information

In [19]:
# Set the random seed
np.random.seed(42)

# List to store mutual information and coefficients for each feature
mutual_info_list = []
coefficient_list = []

# Calculate mutual information and coefficients for each target
for target in y_train.columns:
    # Calculate mutual information
    mutual_info = mutual_info_regression(X_train_numeric, y_train[target])
    mutual_info_list.append(mutual_info)

    # Fit linear regression to get the coefficients
    model = LogisticRegression()
    model.fit(X_train_numeric, y_train[target])
    coefficients = model.coef_
    coefficient_list.append(coefficients)

# Calculate the average mutual information for each feature
average_mutual_info = np.mean(mutual_info_list, axis=0)

# Reshape the coefficient list to be 1-dimensional
coefficient_array = np.array(coefficient_list)
average_coefficients = np.mean(coefficient_array, axis=0).reshape(-1)

# Create a DataFrame with the mutual information, coefficients, and feature names
mutual_info_df = pd.DataFrame({'Feature': X_train_numeric.columns, 'Mutual Information': average_mutual_info, 'Coefficient': average_coefficients})

# Sort the DataFrame by mutual information in descending order
mutual_info_df = mutual_info_df.sort_values('Mutual Information', ascending=False)

# Select the top 10 features
selected_features_mi = mutual_info_df.head(10)

# Print the selected features along with mutual information and coefficients
print(selected_features_mi[['Feature', 'Mutual Information', 'Coefficient']])

                             Feature  Mutual Information  Coefficient
20                 ef_government_soa            0.269571    -0.092041
25              ef_legal_enforcement            0.268133     0.015964
52     ef_regulation_labor_dismissal            0.265102     0.055462
58  ef_regulation_business_impartial            0.257516    -0.037957
26               ef_legal_regulatory            0.232469    -0.033133
24                ef_legal_integrity            0.225595     0.016536
30                  ef_money_sd_data            0.221906    -0.000110
27                   ef_legal_police            0.221587     0.033158
33                 ef_money_currency            0.219600     0.042884
44    ef_regulation_credit_ownership            0.218590     0.040751


Remember, mutual information is a filter method that measures the statistical dependency between two variables, but it doesn't take into account the potential relationships between different features, while RFE is a wrapper method that takes into account the potential relationships between different features by using a specific model to evaluate the importance of each feature. Therefore, these methods can give different results, and using them together can give a more comprehensive view of feature importance.

Feature selection: Exhaustive Feature Selection

In [8]:
# Initialize a random forest regressor
model = RandomForestRegressor(n_jobs=-1)

# Initialize EFS
efs = EFS(model, 
          min_features=1,
          max_features=10,
          scoring='r2',
          print_progress=True,
          cv=2)  

# Fit EFS
#efs = efs.fit(np.array(X_train_numeric), y_train['ef_score'])  # change the target accordingly

# Get the features selected by EFS
selected_features_efs = list(X_train_numeric.columns[list(efs.best_idx_)])

# Print the names of the selected features
print(selected_features_efs)

AttributeError: 'ExhaustiveFeatureSelector' object has no attribute 'best_idx_'

Feature selection: Backward feature selection

In [36]:
import statsmodels.api as sm

# Convert the scaled arrays back to DataFrames because statsmodels.api doesn't work with arrays
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_numeric.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_numeric.columns)

# As we will be using statsmodels, we need to add a constant (intercept) to our model.
X_train_scaled_df = sm.add_constant(X_train_scaled_df)

# We will use all the target variables one by one and find out the significant features for each.
# Then, we will take the intersection of all to get the final set of significant features.

significant_features = []

for target in y_train.columns:
    y = y_train[target].reset_index(drop=True)  # Reset the index here
    
    # Define the model
    model = sm.Logit(y, X_train_scaled_df)
    
    # Fit the model
    result = model.fit(maxiter=100, disp=0)
    
    # Backward Elimination
    cols = list(X_train_scaled_df.columns)
    pmax = 1
    while (len(cols) > 0):
        p = []
        X_1 = X_train_scaled_df[cols]
        model = sm.Logit(y, X_1).fit(disp=0)
        p = pd.Series(model.pvalues.values[1:], index = cols[1:])
        pmax = max(p)
        feature_with_p_max = p.idxmax()
        if(pmax > 0.05):
            cols.remove(feature_with_p_max)
        else:
            break

    significant_features.append(cols)

# Find common significant features among all targets
final_significant_features = set(significant_features[0])
for feature_list in significant_features[1:]:
    final_significant_features.intersection_update(feature_list)

print("The final significant features are:", final_significant_features)


The final significant features are: {'ef_money_currency', 'const', 'ef_regulation_credit', 'pf_identity_divorce', 'ef_regulation_business', 'pf_expression_cld', 'ef_regulation_business_impartial', 'ef_legal_military', 'pf_ss_disappearances', 'pf_ss_disappearances_fatalities'}


## Modelling: Statistical

## Modelling: Genetic

## Modelling: Spatial